目标：

1. 抓取糗事百科热门段子
2. 过滤带有图片的段子
3. 实现每按一次回车显示一个段子的发布时间，发布人，段子内容，点赞数。

糗事百科是不需要登录的，所以也没必要用到Cookie，另外糗事百科有的段子是附图的，我们把图抓下来图片不便于显示，那么我们就尝试过滤掉有图的段子吧。

# 确定URL并抓取页面代码

首先我们确定好页面的URL是 http://www.qiushibaike.com/hot/page/1，其中最后一个数字1代表页数，我们可以传入不同的值来获得某一页的段子内容。

我们初步构建如下的代码来打印页面代码内容试试看，先构造最基本的页面抓取方式，看看会不会成功

In [3]:
# -*- coding:utf-8 -*-
import urllib
import urllib2


page = 1
url = 'http://www.qiushibaike.com/hot/page/' + str(page)
try:
    request = urllib2.Request(url)
    response = urllib2.urlopen(request)
    print response.read()
except urllib2.URLError, e:
    if hasattr(e,"code"):
        print e.code
    if hasattr(e,"reason"):
        print e.reason

BadStatusLine: ''

好吧，应该是headers验证的问题，我们加上一个headers验证试试看吧，将代码修改如下

In [8]:
import urllib
import urllib2
 
page = 1
url = 'http://www.qiushibaike.com/hot/page/' + str(page)
user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
headers = { 'User-Agent' : user_agent }
try:
    request = urllib2.Request(url,headers = headers)
    response = urllib2.urlopen(request)
    print response.read()
except urllib2.URLError, e:
    if hasattr(e,"code"):
        print e.code
    if hasattr(e,"reason"):
        print e.reason











<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="X-UA-Compatible" content="chrome=1,IE=edge">
<meta property="wb:webmaster" content="cb81e216cbe9a3b9" />

<meta name="keywords" content="有趣的,你懂的,很黄很暴力,爆笑囧图"/>
<meta name="description" content="糗事百科笑话段子分享社区,在这里你可以发现校园、办公室、家庭中正在发生的形形色色的搞笑糗事,分享身边有内涵的笑话故事。将无节操无底线进行到底。娱乐自己,放松大家,爆笑态度面对生活。"/>

<title>












【今日头条】 24小时内搞笑爆棚的笑话大全, 搞笑图片 - 糗事百科

</title>
<link href="http://static.qiushibaike.com/css/dist/web/app.min.css?v=5a60e" media="screen, projection" rel="stylesheet" type="text/css" />
<script>
window.app = {}; // global app object init
var _hmt = _hmt || []; // Baidu tongji API
</script>
<script type="text/javascript">
document.createElement('video');document.createElement('audio');document.createElement('track');
</s

# 提取某一页的所有段子

好，获取了HTML代码之后，我们开始分析怎样获取某一页的所有段子。

首先我们审查元素看一下，按浏览器的F12，截图如下

我们可以看到，每一个段子都是''<div class=”article block untagged mb15″ id=”…”>…</div>''包裹的内容。

现在我们想获取发布人，发布日期，段子内容，以及点赞的个数。不过另外注意的是，段子有些是带图片的，如果我们想在控制台显示图片是不现实的，所以我们直接把带有图片的段子给它剔除掉，只保存仅含文本的段子。

所以我们加入如下正则表达式来匹配一下，用到的方法是 re.findall 是找寻所有匹配的内容。方法的用法详情可以看前面说的正则表达式的介绍。

好，我们的正则表达式匹配语句书写如下，在原来的基础上追加如下代码

In [12]:
import re
page = 1
url = 'http://www.qiushibaike.com/hot/page/' + str(page)
user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
headers = { 'User-Agent' : user_agent }
try:
    request = urllib2.Request(url,headers = headers)
    response = urllib2.urlopen(request)
    content = response.read().decode('utf-8')
    pattern = re.compile('<div.*?class="author.*?>.*?<a.*?</a>.*?<a.*?>(.*?)</a>.*?<div.*?class'+
                     '="content".*?title="(.*?)">(.*?)</div>(.*?)<div class="stats.*?class="number">(.*?)</i>',re.S)
    items = re.findall(pattern,content)
    for item in items:
        print item[0],item[1],item[2],item[3],item[4]
except urllib2.URLError, e:
    if hasattr(e,"code"):
        print e.code
    if hasattr(e,"reason"):
        print e.reason
        


空虚！寂寞！冷！  84条评论" target="_blank" onclick="_hmt.push(['_trackEvent', 'post', 'click', 'signlePost']) 
<i class="number">84</i> 回复
</a>



</span>

<div id="qiushi_counts_106162292" class="stats-buttons bar clearfix">
<ul class="clearfix">
<li id="vote-up-106162292" class="up">
<a href="javascript:voting(106162292,1)" class="voting" data-article="106162292" id="up-106162292" title="16913个顶">
<i class="iconfont" data-icon-actived="&#xf0061;" data-icon-original="&#xf001f;">&#xf001f;</i>
<span class="number hidden">16913</span>
</a>
</li>
<li id="vote-dn-106162292" class="down">
<a href="javascript:voting(106162292,-1)" class="voting" data-article="106162292" id="dn-106162292" title="-298个拍">
<i class="iconfont" data-icon-actived="&#xf0020;" data-icon-original="&#xf0020;">&#xf0020;</i>
<span class="number hidden">-298</span>
</a>
</li>

<li class="comments">
<a href="/article/106162292?list=hot&s=4771464" id="c-106162292" class="qiushi_comments" title="84条评论" target="_blank" onclick="_hmt.p

在正则表达式在这里稍作说明

1）.*? 是一个固定的搭配，.和*代表可以匹配任意无限多个字符，加上？表示使用非贪婪模式进行匹配，也就是我们会尽可能短地做匹配，以后我们还会大量用到 .*? 的搭配。

2）(.*?)代表一个分组，在这个正则表达式中我们匹配了五个分组，在后面的遍历item中，item[0]就代表第一个(.*?)所指代的内容，item[1]就代表第二个(.*?)所指代的内容，以此类推。

3）re.S 标志代表在匹配时为点任意匹配模式，点 . 也可以代表换行符。

这是其中的两个段子，分别打印了发布人，发布时间，发布内容，附加图片以及点赞数。

其中，附加图片的内容我把图片代码整体抠了出来，这个对应item[3]，所以我们只需要进一步判断item[3]里面是否含有img这个字样就可以进行过滤了。

好，我们再把上述代码中的for循环改为下面的样子

In [11]:
for item in items:
        haveImg = re.search("img",item[3])
        if not haveImg:
            print item[0],item[1],item[2],item[4]

''

现在，整体的代码如下

In [1]:
# -*- coding:utf-8 -*-
import urllib
import urllib2
import re

page = 1
url = 'http://www.qiushibaike.com/hot/page/' + str(page)
user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
headers = { 'User-Agent' : user_agent }
try:
    request = urllib2.Request(url,headers = headers)
    response = urllib2.urlopen(request)
    content = response.read().decode('utf-8')
    pattern = re.compile('<div.*?class="author">.*?<a.*?</a>.*?<a.*?>(.*?)</a>.*?<div.*?class="content">(.*?)<!--(.*?)-->.*?</div>.*?<div.*?class="stats">.*?<span.*?>.*?<i.*?>(.*?)</i>',re.S)
    items = re.findall(pattern,content)
    for item in items:
        haveImg = re.search("img",item[2])
        if not haveImg:
            print item[0],item[1],item[2],item[3]
except urllib2.URLError, e:
    if hasattr(e,"code"):
        print e.code
    if hasattr(e,"reason"):
        print e.reason

枫叶:.  

支持的默默点赞！
2015-05-14 08:59:30 9369
画地为牢囚禁心  

圆珠笔手绘，纯手工。小蛋壳又回归咯
2015-05-14 15:13:03 8266
热电多用  

前两天去银行取钱，叫到一个姓王的，结果来的好几个，抬了一张小桌子，上面摆了好几件法器，还有一道士，大堂经理赶紧上前一问，原来是他家老头过世后才发现有一张存折沒取，正按银行规定叫他本人来呢!
2015-05-14 08:56:53 14882
零度冰吻  

孩子加油，坚强的活下去，大难不死必有后福。
2015-05-13 11:13:18 3477
天苍苍呀野茫茫  

今天早起突然下起小雨，吃过早餐去上班，二货老婆不想打自己的雨伞，马上凑到我身边说：嗨！帅哥我们一起打个伞可以吗？我非常认真的说：不行 。旁边有个大叔鄙视的眼神看着我，温柔的对我媳妇说：美女到我伞里来，我顿时觉悟了。
2015-05-14 12:17:15 6907
秋雨6393  

9月3日，为纪念世界反法西斯战争胜利70周年，全国放假一天，我就在想了，日企的朋友们怎么办？尤其是HR，怎么跟老板去说呢？脑补这样的场景:太君，八路托我给您带个话儿，今天是您败给我们的日子，我得回去庆祝一下。。。
2015-05-14 18:10:53 6363
暖风吹的屁屁凉  

纯手工钩织的，大神给过来吧~
2015-05-14 08:57:05 1427
妹子的妹子de  

（朋友发的微博）刚和我妈打车去大昭寺，我们问司机拉萨当地的治安怎么样，司机说拉萨治安特别好，我们猜可能是因为警力多吧，然后司机：不，是因为小偷跑路跑两步就会高原反应大喘气跑不动了……噗……
2015-05-14 18:11:57 4482
经年未变ii  

各位糗友们注意了，现在新的骗术又出来了，骗子冒充淘宝网订单处理中心的，说是你买的东西订单异常，让你实名认证后退款，来套取你的个人信息，然后你的银行卡里就莫名的少钱了。请各位顶上去让大家都警惕，防止被骗。
2015-05-14 16:23:33 4469
不约~而同  

今早撞了一辆摩托车，从车头爬起来一个身穿制服的人，我蒙了，没错，是个交警，还是个正在执勤的交警~~<br/>“小伙儿挺猛啊，我干了十多年交警，敢撞我的你算头一个”<br/>“哥~…”<br/>“行驶证…驾驶证…

In [25]:
content

u'\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<meta http-equiv="X-UA-Compatible" content="chrome=1,IE=edge">\n<meta property="wb:webmaster" content="cb81e216cbe9a3b9" />\n\n<meta name="keywords" content="\u6709\u8da3\u7684,\u4f60\u61c2\u7684,\u5f88\u9ec4\u5f88\u66b4\u529b,\u7206\u7b11\u56e7\u56fe"/>\n<meta name="description" content="\u7cd7\u4e8b\u767e\u79d1\u7b11\u8bdd\u6bb5\u5b50\u5206\u4eab\u793e\u533a,\u5728\u8fd9\u91cc\u4f60\u53ef\u4ee5\u53d1\u73b0\u6821\u56ed\u3001\u529e\u516c\u5ba4\u3001\u5bb6\u5ead\u4e2d\u6b63\u5728\u53d1\u751f\u7684\u5f62\u5f62\u8272\u8272\u7684\u641e\u7b11\u7cd7\u4e8b,\u5206\u4eab\u8eab\u8fb9\u6709\u5185\u6db5\u7684\u7b11\u8bdd\u6545\u4e8b\u3002\u5c06\u65e0\u8282\u64cd\u65e0\u5e95\u7ebf\u8fdb\u884c\u5230\u5e95\u3002\u5a31\u4e50\u81ea\u5df1,

# 完善交互，设计面向对象模式

好啦，现在最核心的部分我们已经完成啦，剩下的就是修一下边边角角的东西，我们想达到的目的是：

按下回车，读取一个段子，显示出段子的发布人，发布日期，内容以及点赞个数。

另外我们需要设计面向对象模式，引入类和方法，将代码做一下优化和封装，最后，我们的代码如下所示



In [35]:
__author__ = 'CQC'
# -*- coding:utf-8 -*-
import urllib
import urllib2
import re
import thread
import time

#糗事百科爬虫类
class QSBK:

    #初始化方法，定义一些变量
    def __init__(self):
        self.pageIndex = 1
        self.user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
        #初始化headers
        self.headers = { 'User-Agent' : self.user_agent }
        #存放段子的变量，每一个元素是每一页的段子们
        self.stories = []
        #存放程序是否继续运行的变量
        self.enable = False
    #传入某一页的索引获得页面代码
    def getPage(self,pageIndex):
        try:
            url = 'http://www.qiushibaike.com/hot/page/' + str(pageIndex)
            #构建请求的request
            request = urllib2.Request(url,headers = self.headers)
            #利用urlopen获取页面代码
            response = urllib2.urlopen(request)
            #将页面转化为UTF-8编码
            pageCode = response.read().decode('utf-8')
            return pageCode

        except urllib2.URLError, e:
            if hasattr(e,"reason"):
                print u"连接糗事百科失败,错误原因",e.reason
                return None


    #传入某一页代码，返回本页不带图片的段子列表
    def getPageItems(self,pageIndex):
        pageCode = self.getPage(pageIndex)
        if not pageCode:
            print "页面加载失败...."
            return None
        pattern = re.compile('<div.*?class="author">.*?<a.*?</a>.*?<a.*?>(.*?)</a>.*?<div.*?class="content">(.*?)<!--(.*?)-->.*?</div>.*?<div.*?class="stats">.*?<span.*?>.*?<i.*?>(.*?)</i>',re.S)
        items = re.findall(pattern,pageCode)
        #用来存储每页的段子们
        pageStories = []
        #遍历正则表达式匹配的信息
        for item in items:
            #是否含有图片
            haveImg = re.search("img",item[3])
            #如果不含有图片，把它加入list中
            if not haveImg:
                #item[0]是一个段子的发布者，item[1]是发布时间,item[2]是内容，item[4]是点赞数
                pageStories.append([item[0].strip(),item[1].strip(),item[2].strip(),item[3].strip()])
        return pageStories

    #加载并提取页面的内容，加入到列表中
    def loadPage(self):
        #如果当前未看的页数少于2页，则加载新一页
        if self.enable == True:
            if len(self.stories) < 2:
                #获取新一页
                pageStories = self.getPageItems(self.pageIndex)
                #将该页的段子存放到全局list中
                if pageStories:
                    self.stories.append(pageStories)
                    #获取完之后页码索引加一，表示下次读取下一页
                    self.pageIndex += 1
    
    #调用该方法，每次敲回车打印输出一个段子
    def getOneStory(self,pageStories,page):
        #遍历一页的段子
        for story in pageStories:
            #等待用户输入
            input = raw_input()
            #每当输入回车一次，判断一下是否要加载新页面
            self.loadPage()
            #如果输入Q则程序结束
            if input == "Q":
                self.enable = False
                return
            print u"第%d页\t发布人:%s\t发布时间:%s\n%s\n赞:%s\n" %(page,story[0],story[1],story[2],story[3])
    
    #开始方法
    def start(self):
        print u"正在读取糗事百科,按回车查看新段子，Q退出"
        #使变量为True，程序可以正常运行
        self.enable = True
        #先加载一页内容
        self.loadPage()
        #局部变量，控制当前读到了第几页
        nowPage = 0
        while self.enable:
            if len(self.stories)>0:
                #从全局list中获取一页的段子
                pageStories = self.stories[0]
                #当前读到的页数加一
                nowPage += 1
                #将全局list中第一个元素删除，因为已经取出
                del self.stories[0]
                #输出该页的段子
                self.getOneStory(pageStories,nowPage)


spider = QSBK()
spider.start()

正在读取糗事百科,按回车查看新段子，Q退出

第1页	发布人:空虚！寂寞！冷！	发布时间:过了的！明天都有好事！
2015-05-12 06:23:18
赞:20375


第1页	发布人:罗马不停蹄	发布时间:同事网上花5块钱买了条假的粗金链子，人品爆发，卖家送了9条。结果同事送给办公室男同事每人一条，全都带上了…那场面，把领导直接镇住了
2015-05-11 14:34:01
赞:9837


第1页	发布人:冷子宜2	发布时间:小时候骑着爸爸的大凤凰车，带着弟弟去江边偷桃子，被发现了，然后带着弟弟飞快逃走，感觉安全时，下车一看弟弟没了，只好硬着头皮往回找，结果在半路看见弟弟被树枝勾那在嚎呢，真事！
2015-05-12 10:39:45
赞:9535


第1页	发布人:我罩着你	发布时间:“大夫，我上次刚打完疫苗，刚才又被狗咬了，还用打疫苗吗？”<br/>“怎么又被狗咬了？”<br/>“噢，是我找它去报仇了”<br/>“不用打疫苗了，不过我建议你去精神科看一看”
2015-05-12 08:50:42
赞:6957


第1页	发布人:雲想衣裳花想容N	发布时间:今天做公交车，上来一女的，穿的比较暴漏，冻得跟狗一样。进了车里这货就说，车里真暖和，像被窝里一样，旁边一大姐说，你家被窝里有这么多人？只听车里传来一片笑声。
2015-05-12 07:58:59
赞:8492


第1页	发布人:哈库纳玛踏踏	发布时间:不过没道理
2015-05-12 09:06:43
赞:4691


第1页	发布人:烂泥扶不上墙啊呀	发布时间:员工办离职，需要写离职报告，偌大一张纸，他只写4个字：“不想干了。” 人事经理摇摇头，把报告递还给他说：“你就不能多写点？” 不一会儿，员工又来交报告了。人事经理拿过来一看，“不想干了”前面多了两个字：“真的！”
2015-05-12 04:28:31
赞:5073

Q
